# §2.3 Symbolic Data

### Exercise 2.53

In [1]:
(list 'a 'b 'c)

(a b c)

In [2]:
(list (list 'george))

((george))

In [3]:
(cdr '((x1 x2) (y1 y2)))

((y1 y2))

In [4]:
(cadr '((x1 x2) (y1 y2)))

(y1 y2)

In [5]:
(pair? (car '(a short list))) ; ~~> (pair? 'a) thus #f

#f

In [6]:
(memq 'red '((red shoes) (lue socks))) ; memq doesn't deal with nested list, thus #f

#f

In [7]:
(memq 'red '(red shoes blue socks))

(red shoes blue socks)

### Exercise 2.54

In [8]:
(define (my-equal? xs ys)
  (cond
    [(and (pair? xs) (pair? ys))
     (and (my-equal? (car xs) (car ys))
          (my-equal? (cdr xs) (cdr ys)))]
    [else (eq? xs ys)]))

In [9]:
(list
 (my-equal? 'a 'a)
 (my-equal? 'a 'b)
 (my-equal? 3 3)
 (my-equal? 4 3))

(#t #f #t #f)

In [10]:
(list
 (my-equal? '(a b c) '(a b c))
 (my-equal? '(a (b) c) '(a (b) c))
 (my-equal? '(a (b) c) '(a (b d) c))
 (my-equal? '(a b c) 'a)
 (my-equal? '(a b c) '(b c a)))

(#t #t #f #f #f)

### Exercise 2.55

In [11]:
(car ''abracadabra) ; ~~> (car '(quote abracadabra)) ~~> 'quote

quote

## §§2.3.2 Example: Symbolic Differentiation

In [12]:
(if #t 0
  (list ; we just wish that we have the following; we just wish...
   (variable? e)
   (same-variable? v1 v2)
   
   (sum? e)
   (addend e)
   (augend e)
   (make-sum a1 a2)
   
   (product? e)
   (multiplier e)
   (multiplicant e)
   (make-product m1 m2)))

0

In [13]:
(define (deriv exp var)
  (cond [(number? exp) 0]
        [(variable? exp)
         (if (same-variable? exp var) 1 0)]
        [(sum? exp)
         (make-sum (deriv (addend exp) var)
                   (deriv (augend exp) var))]
        [(product? exp)
         (make-sum
          (make-product (multiplier exp)
                        (deriv (multiplicand exp) var))
          (make-product (multiplicand exp)
                        (deriv (multiplier exp) var)))]
        [else (error "what are you feeding me? is that dog food?! -- DERIV") exp]))

In [14]:
;; we use symbols to represent variables
(define (variable? e) (symbol? e))
(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))

;; we use lists of the form ('+ e1 e2) to represent sums
(define (sum? e)
  (and (pair? e) (eq? (car e) '+)))
(define (make-sum a1 a2) (list '+ a1 a2))
(define (addend e) (cadr e))
(define (augend e) (caddr e))

;; we use lists of the form ('* e1 e2) to represent products
(define (product? e)
  (and (pair? e) (eq? (car e) '*)))
(define (make-product m1 m2) (list '* m1 m2))
(define (multiplier e) (cadr e))
(define (multiplicand e) (caddr e))

In [15]:
(deriv '(+ x 3) 'x)

(+ 1 0)

In [16]:
(deriv '(* (* x y) (+ x 3)) 'x)

(+ (* (* x y) (+ 1 0)) (* (+ x 3) (+ (* x 0) (* y 1))))

In [17]:
(define (make-sum a1 a2)
  (cond [(and (number? a1) (number? a2)) (+ a1 a2)]
        [(eqv? 0 a1) a2]
        [(eqv? 0 a2) a1]
        [else (list '+ a1 a2)]))

(define (make-product m1 m2)
  (cond [(and (number? m1) (number? m2)) (* m1 m2)]
        [(eqv? 1 m1) m2]
        [(eqv? 1 m2) m1]
        [(or (eqv? 0 m1) (eqv? 0 m2)) 0]
        [else (list '* m1 m2)]))

In [18]:
(deriv '(+ x 3) 'x)

1

In [19]:
(deriv '(* x y) 'x)

y

In [20]:
(deriv '(* (* x y) (+ x 3)) 'x)

(+ (* x y) (* (+ x 3) y))

In [21]:
(deriv '(* (* x y) (+ x 3)) 'x)

(+ (* x y) (* (+ x 3) y))

### Exercise 2.56

In [22]:
(define (deriv exp var)
  (cond [(number? exp) 0]
        [(variable? exp)
         (if (same-variable? exp var) 1 0)]
        [(sum? exp)
         (make-sum (deriv (addend exp) var)
                   (deriv (augend exp) var))]
        [(product? exp)
         (make-sum
          (make-product (multiplier exp)
                        (deriv (multiplicand exp) var))
          (make-product (multiplicand exp)
                        (deriv (multiplier exp) var)))]
        [(exponentiation? exp)
         (make-product (exponent exp)
                       (make-product (make-exponentiation (base exp) (- (exponent exp) 1))
                                     (deriv (base exp) var)))]
        [else (error "what are you feeding me? is that dog food?! -- DERIV") exp]))

;; we use lists of form '(** x a) to express exponentiations.
;; we restrict a to be a number
(define (exponentiation? e) (and (pair? e) (eq? (car e) '**)))
(define (base e) (cadr e))
(define (exponent e) (caddr e))
(define (make-exponentiation x a)
  (cond [(eqv? x 0) 0]
        [(eqv? x 1) 1]
        [(eqv? a 0) 1]
        [(eqv? a 1) x]
        [else (list '** x a)]))

In [23]:
(deriv '(** x 3) 'x)

(* 3 (** x 2))

In [24]:
(deriv '(** (+ (** x 6) (* 2 x)) 3) 'x)

(* 3 (* (** (+ (** x 6) (* 2 x)) 2) (+ (* 6 (** x 5)) 2)))

### Exercise 2.57

In [25]:
%include "/library/prelude.scm"
;; we are going to use `fn` and `rec` defined there
;; the file also defines some high-order list functions

#### Straightforward implementation

Since we know that

\begin{align}
\frac{\partial}{\partial x} (f_1(x) + f_2(x) + \cdots + f_n(x))
  \; = \; & f_1'(x) + f_2'(x) + \cdots + f_n'(x) \\
\frac{\partial}{\partial x} (f_1(x) \cdot f_2(x) \cdot \cdots \cdot f_n(x))
  \; = \; & f_1'(x) \cdot f_2(x) \cdot \cdots \cdot f_n(x) \\
     + \; & f_2'(x) \cdot f_1(x) \cdot \cdots \cdot f_n(x) \\
     + \; & \cdots \\
     + \; & f_n'(x) \cdot f_1(x) \cdot f_2(x) \cdot \cdots \cdot f_{n-1}(x)
\end{align}

It's quite straightforward to re-write the `deriv` function using higher order functions on lists (e.g. `map`).

The trick is that we want a higher order function in implementing the derivative of products that pick each element (with the else of the list) from a list, which we call it `pick` (and a corresponding map-function, `pick-map`) and will implement it later.

In [26]:
(define (deriv exp var)
  (define (D e) (deriv e var))
  (cond [(number? exp) 0]
        [(variable? exp)
         (if (same-variable? exp var) 1 0)]
        [(sum? exp)
         (make-sum (map D (terms exp)))]
        [(product? exp)
         (make-sum
          (pick-map (fn (e es) (make-product (cons (D e) es)))
                    (terms exp)))]
        [(exponentiation? exp)
         (make-product-lit (exponent exp)
                           (make-product-lit
                             (make-exponentiation (base exp) (- (exponent exp) 1))
                             (deriv (base exp) var)))]
        [else (error "what are you feeding me? is that dog food?! -- DERIV") exp]))

;; we use lists of the form ('+ e1 e2 ...) to represent sums
(define (sum? e)
  (and (pair? e) (eq? (car e) '+)))
(define (make-sum terms) (cons '+ terms))
(define (make-sum-lit a1 a2) (make-sum (list a1 a2)))
(define (terms e) (cdr e))

;; we use lists of the form ('* e1 e2 ...) to represent products
(define (product? e)
  (and (pair? e) (eq? (car e) '*)))
(define (make-product factors) (cons '* factors))
(define (make-product-lit m1 m2) (make-product (list m1 m2)))
(define (factors e) (cdr e))

#### `pick` and `pick-map`

In [27]:
(define (pick xs)
  (define (F front tail acc)
    (if (null? tail) acc
        (F (cons (car tail) front)
           (cdr tail)
           (cons (cons (car tail) (append (reverse front) (cdr tail))) acc))))
  (reverse (F '() xs '())))
(define (pick-map f xs)
  (map (fn (p) (f (car p) (cdr p))) (pick xs)))

In [28]:
(pick (range 5))

((0 1 2 3 4) (1 0 2 3 4) (2 0 1 3 4) (3 0 1 2 4) (4 0 1 2 3))

In [29]:
(pick-map (fn (x xs) (cons xs x)) (range 5))

(((1 2 3 4) . 0) ((0 2 3 4) . 1) ((0 1 3 4) . 2) ((0 1 2 4) . 3) ((0 1 2 3) . 4))

#### Refinements of constructors `make-sum` and `make-product`

In [30]:
(define (make-sum terms)
  (define filtered (filter-out (eqv-pred 0) terms))
  (cond [(null? filtered) 0]
        [(forall number? terms) (sum filtered)]
        [(singleton? filtered) (car filtered)]
        [else (cons '+ filtered)]))
(define (make-product factors)
  (define (prod xs) (foldl * 1 xs))
  (define filtered (filter-out (eqv-pred 1) factors))
  (cond [(null? filtered) 1]
        [(forall number? filtered) (prod filtered)]
        [(exists (eqv-pred 0) filtered) 0]
        [(singleton? filtered) (car filtered)]
        [else (cons '* filtered)]))

(define (filter-out pred xs) (filter (negate pred) xs))
(define (eqv-pred x) (fn (y) (eqv? x y)))

#### Tryouts

In [31]:
(deriv '(** (+ (** x 6) (* 2 x)) 3) 'x)

(* 3 (* (** (+ (** x 6) (* 2 x)) 2) (+ (* 6 (** x 5)) 2)))

In [32]:
(deriv '(+ (* 3 x) (* 4 x) (* (** x 2) y)) 'x)

(+ 3 4 (* (* 2 x) y))

In [33]:
(deriv '(* (+ (* 3 x) (* 4 x) (* (** x 2) y)) y x) 'x)

(+ (* (+ 3 4 (* (* 2 x) y)) y x) (* (+ (* 3 x) (* 4 x) (* (** x 2) y)) y))

### Exercise 2.58 (a)

The key for this exercise is the `interpose` higher-order list function. What it does is to insert a /separator/ in between of every consecutive pairs of elements in a list.

E.g.
````
(interpose '+ '(1 2 3)) ~~> '(1 + 2 + 3)
(interpose '$ '(4 (3 2) 1)) ~~> '(4 $ (3 2) $ 1)
````

Of course you also need to realize that the `algebraic->infix` function is recursive, in the sense that every subexpression of an algebraic expression should also be converted to the infix form. This is obvious since you should have noticed that the algebraic expressions are structurally trees.

In [34]:
(define (algebraic->infix e)
  (cond [(number? e) e]
        [(variable? e) e]
        [(sum? e) (interpose '+ (map algebraic->infix (terms e)))]
        [(product? e) (interpose '* (map algebraic->infix (factors e)))]
        [(exponentiation? e) (list (base e) '** (exponent e))]))

(define (deriv-pp e var) (algebraic->infix (deriv e var)))

In [35]:
(deriv-pp '(+ (* 3 x) (* 4 x) (* (** x 2) y)) 'x)

(3 + 4 + ((2 * x) * y))

In [36]:
(deriv-pp '(* (+ (* 3 x) (* 4 x) (* (** x 2) y)) y x) 'x)

(((3 + 4 + ((2 * x) * y)) * y * x) + (((3 * x) + (4 * x) + ((x ** 2) * y)) * y))

### Exercise 2.58 (b)

Actually this is known as the "parsing problem". It is known to be possible to parse infix algebraic expressions and the language of those expressions are known to be in the Context Free Language class.

An implementation for the parser is skipped here for now. If you're interested, consult the Internet.